In [1]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.google_bard import BardTextCompletion
from semantic_kernel.connectors.ai.sentence_transformers import SentenceTransformersTextEmbedding
from semantic_kernel.connectors.memory.chroma.chroma_memory_store import ChromaMemoryStore

c:\Users\yashw\AppData\Local\pypoetry\Cache\virtualenvs\semantic-kernel-MznNScXN-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kernel = sk.Kernel()

api_key = sk.bard_settings_from_dot_env()
kernel.add_text_completion_service("dv", BardTextCompletion(api_key))
kernel.add_text_embedding_generation_service("ada", SentenceTransformersTextEmbedding('all-mpnet-base-v2'))

kernel.register_memory_store(memory_store=ChromaMemoryStore('./chroma/memory'))
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1dc5f2dde40>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1dc5f2ddf90>}

In [20]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(
        "aboutMe", id="info1", text="My name is Andrea"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info2", text="I currently work as a tour guide"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info4", text="I visited France and Italy five times since 2015"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info5", text="My family is from New York"
    )

In [3]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async("aboutMe", question)
        print(f"Answer: {result[0].text}\n")

In [22]:
async def setup_chat_with_memory(
    kernel: sk.Kernel,
) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.

    Information about me, from previous conversations:
    - {{$fact1}} {{recall $fact1}}
    - {{$fact2}} {{recall $fact2}}
    - {{$fact3}} {{recall $fact3}}
    - {{$fact4}} {{recall $fact4}}
    - {{$fact5}} {{recall $fact5}}

    Chat:
    {{$chat_history}}
    User: {{$user_input}}
    ChatBot: """.strip()

    chat_func = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0.8)

    context = kernel.create_new_context()
    context["fact1"] = "what is my name?"
    context["fact2"] = "where do I live?"
    context["fact3"] = "where's my family from?"
    context["fact4"] = "where have I traveled?"
    context["fact5"] = "what do I do for work?"

    context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "aboutMe"
    context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8

    context["chat_history"] = ""

    return chat_func, context

In [23]:
async def chat(
    kernel: sk.Kernel, chat_func: sk.SKFunctionBase, context: sk.SKContext
) -> bool:
    try:
        user_input = input("User:> ")
        context["user_input"] = user_input
        print(f"User:> {user_input}")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False

    answer = await kernel.run_async(chat_func, input_vars=context.variables)
    context["chat_history"] += f"\nUser:> {user_input}\nChatBot:> {answer}\n"

    print(f"ChatBot:> {answer}")
    return True

In [4]:
print("Populating memory...")
await populate_memory(kernel)

print("Asking questions... (manually)")
await search_memory_examples(kernel)

print("Setting up a chat (with memory!)")
chat_func, context = await setup_chat_with_memory(kernel)

print("Begin chatting (type 'exit' to exit):\n")
chatting = True
while chatting:
    chatting = await chat(kernel, chat_func, context)

Asking questions... (manually)
Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: My family is from New York

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I visited France and Italy five times since 2015

Question: what do I do for work
Answer: I currently work as a tour guide



In [5]:
memories = await kernel.memory.search_async("aboutMe", "Do you know where I stay?", limit=5, min_relevance_score=0.1)

i = 0
for memory in memories:
    i += 1
    print(f"Result {i}:")
    print("  Id:      : " + memory.id)
    print("  Text     : " + memory.text)
    print("  Relevance: " + str(memory.relevance))
    print()

Result 1:
  Id:      : info1
  Text     : My name is Andrea
  Relevance: 0.24663701388418288

Result 2:
  Id:      : info3
  Text     : I've been living in Seattle since 2005
  Relevance: 0.22820872870880163

Result 3:
  Id:      : info2
  Text     : I currently work as a tour guide
  Relevance: 0.20848871460323137

Result 4:
  Id:      : info5
  Text     : My family is from New York
  Relevance: 0.18716519257499822

Result 5:
  Id:      : info4
  Text     : I visited France and Italy five times since 2015
  Relevance: 0.1640274834122064



In [6]:
github_files ={}
github_files["https://github.com/microsoft/semantic-kernel/blob/main/README.md"] = \
    "README: Installation, getting started, and how to contribute"
github_files["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb"] = \
    "Jupyter notebook describing how to pass prompts from a file to a semantic skill or function"
github_files["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb"] = \
    "Jupyter notebook describing how to get started with the Semantic Kernel"
github_files["https://github.com/microsoft/semantic-kernel/tree/main/samples/skills/ChatSkill/ChatGPT"] = \
    "Sample demonstrating how to create a chat skill interfacing with ChatGPT"
github_files["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs"] = \
    "C# class that defines a volatile embedding store"
github_files["https://github.com/microsoft/semantic-kernel/tree/main/samples/dotnet/KernelHttpServer/README.md"] = \
    "README: How to set up a Semantic Kernel Service API using Azure Function Runtime v4"
github_files["https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md"] = \
    "README: README associated with a sample starter react-based chat summary webapp"

In [27]:
memory_collection_name = "SKGitHub"
print("Adding some GitHub file URLs and their descriptions to a Chroma Semantic Memory.");
i = 0
for entry, value in github_files.items():
    await kernel.memory.save_reference_async(
        collection=memory_collection_name,
        description=value,
        text=value,
        external_id=entry,
        external_source_name="GitHub"
    )
    i += 1
    print("  URL {} saved".format(i))

Adding some GitHub file URLs and their descriptions to a Chroma Semantic Memory.
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved
  URL 5 saved
  URL 6 saved
  URL 7 saved


In [7]:
ask = "I love Jupyter notebooks, how should I get started?"
print("===========================\n" + "Query: " + ask + "\n")

memories = await kernel.memory.search_async(memory_collection_name, ask, limit=5, min_relevance_score=0.4)

i = 0
for memory in memories:
    i += 1
    print(f"Result {i}:")
    print("  URL:     : " + memory.id)
    print("  Title    : " + memory.description)
    print("  Relevance: " + str(memory.relevance))
    print()

Query: I love Jupyter notebooks, how should I get started?

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.570058292917489

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic skill or function
  Relevance: 0.41668696409585926

